In [ ]:
from scipy.stats import gamma, multivariate_normal
from random import sample
import numpy

from matplotlib import pyplot as plot
from tqdm.notebook import tqdm
from itertools import product

%matplotlib inline

In [ ]:
class grid_world():
    def __init__(self, length, epochs):
        self.spaces = numpy.array(list( product( range(length), repeat = 2) ))
        self.epochs = epochs
        
    def generate_densities(self, users):        
        centers = numpy.random.randint(0, len(self.spaces), size = users)
        centers = self.spaces[ centers ]
        
        distances = numpy.repeat(self.spaces[:, :, None], users, axis = 2) - centers.T
        distances = ( numpy.sqrt( numpy.sum( distances ** 2, axis = 1) ) + 1 ) ** 3
        
        initial = ( numpy.random.uniform(0, 1, size = distances.shape) / distances )      
        initial = numpy.round( initial / numpy.sum( initial, axis = 0 ), decimals = 3 )
        
        return (initial / numpy.sum( initial, axis = 0 )).T

In [ ]:
class user():
    def __init__(self, world, density):
        self.world = world
        self.location_density = density
        self.location_history = self.sample(size = world.epochs )
        
    def sample(self, size = 1):
        indices = numpy.random.choice( len(self.world.spaces), p = self.location_density, size = size )
        return self.world.spaces[indices]

In [ ]:
# Prior: Could be anything

def movement_pdf(distances):
    distances = numpy.sqrt( numpy.sum( distances ** 2, axis = 1) )
    non_normalized = 0.5 / ((distances + 1) ** 3)
    return non_normalized

In [ ]:
class cloaking():
    def __init__(self, epsilon):
        self.side_length = round( ((12 / epsilon) + 1) ** 0.25 )
        
    def noise(self, locations):
        locations = locations // self.side_length
        return ( locations * self.side_length ) + numpy.array([ self.side_length / 2, self.side_length / 2 ])
    
    def pdf(self, distances):
        indicators = numpy.all( abs(distances) < self.side_length, axis = 1 )
        return indicators / (self.side_length ** 2)

In [ ]:
class geo_indistinguishable():
    def __init__(self, epsilon):
        self.epsilon = epsilon
        
    def noise(self, locations):
        return locations + self.sample_noise( locations.shape[0:2] )
    
    def sample_noise(self, size):
        theta = numpy.random.uniform(0, 2 * numpy.pi, size = size)
        r = gamma.ppf( numpy.random.uniform(0, 1, size = size), 2, scale = 1 / self.epsilon )        
        
        x, y = r * numpy.cos(theta), r * numpy.sin(theta)
        return numpy.stack([x, y], axis = 2)
    
    def pdf(self, distances):
        distances = (- self.epsilon) * numpy.sqrt( numpy.sum( distances ** 2, axis = 1) )
        return ((self.epsilon ** 2) / (2 * numpy.pi)) * numpy.exp( distances )

In [ ]:
class gaussian():
    def __init__(self, epsilon):
        self.random_variable = multivariate_normal(mean = [0, 0], cov = numpy.eye(2) / epsilon)
        
    def noise(self, locations):
        return locations + self.random_variable.rvs( size = locations.shape[:-1] )
    
    def pdf(self, distances):
        return numpy.round( self.random_variable.pdf( distances ), decimals = 10 )

In [ ]:
privacy_schemes = {
    "Gaussian": gaussian,
    "Geo-Indistinguishability": geo_indistinguishable,
    "Cloaking": cloaking
}

In [ ]:
TRIALS = 10
RADIUS = 10     # radius in kilometers

In [ ]:
class server():
    def __init__(self, world, users, scheme):
        self.world, self.users, self.scheme = world, users, scheme
        
        self.locations  = numpy.array([ user.location_history for user in self.users ])
        self.heartbeats = self.scheme.noise( self.locations )
        # self.heartbeats[ user, epoch, coordinate ]
        
    def true_consensus(self, user, epoch):
        center = self.locations[user, epoch, :]
        
        timestamp = self.locations[:, epoch, :]
        distances = numpy.sqrt( numpy.sum(timestamp ** 2, axis = 1) ).astype(int)
        return set([ i for i in range(len(distances)) if distances[i] < RADIUS ])
        
    def private_consensus(self, user, epoch):
        center = self.heartbeats[user, epoch, :]
        
        timestamp = self.heartbeats[:, epoch, :]
        distances = numpy.sqrt( numpy.sum(timestamp ** 2, axis = 1) ).astype(int)
        return set([ i for i in range(len(distances)) if distances[i] < RADIUS ])
    
    def accuracy(self):
        instances = product( range(len(self.users)), range(self.world.epochs) )
        overlap, instances = 0, sample( list(instances), TRIALS )

        for user, epoch in instances:
            actual  = self.true_consensus(user, epoch)
            private = self.private_consensus(user, epoch)
            
            results = len(actual.intersection( private )) / (len(actual.union( private )) + 1)            
            overlap = overlap + results
        
        return overlap / TRIALS

In [ ]:
class semi_honest(server):
    def inference(self, user, latest, prior):
        conditional_noise  = self.scheme.pdf( self.heartbeats[user, latest, :] - self.world.spaces )
        return prior + conditional_noise
    
    def timeseries_inference(self, user):
        initial = numpy.zeros(( len(self.world.spaces), ))
        estimates = [ initial ]
        
        for i in range(self.world.epochs):
            estimates.append( self.inference(user, i, estimates[i]) )
            center = numpy.mean( self.heartbeats[user, :i + 1, :], axis = 0 )
            estimates[i] = movement_pdf( self.world.spaces - center ) * estimates[i]
            
        estimates = numpy.array(estimates) / ( numpy.sum(estimates, axis = 1)[:, None] + (0.1 ** 11) )
        estimates = numpy.round( estimates[:-1], decimals = 10 )
        return estimates
    
    def bhattacharyya(self, estimate, true):
        return numpy.sum(numpy.sqrt(estimate * true), axis = 1)
        
    def average_predictive_power(self):
        predictive_power = numpy.zeros(( self.world.epochs, ))
        
        for user in tqdm(sample( range(len(self.users)), TRIALS ), desc = "Trials"):
            predicted = self.timeseries_inference(user)
            predictive_power = predictive_power + self.bhattacharyya(predicted, self.users[user].location_density)
        
        predictive_power = predictive_power / TRIALS
        return predictive_power

In [ ]:
numpy.random.seed(12345)

In [ ]:
bay_area = grid_world(134, 700)
# 18000 (~ 134^2) km^2, two weeks (~ 700 heartbeat messages)

population = 7000
# 1 per 1,000 inhabitants

movement = bay_area.generate_densities(population)
go_bears = numpy.array([ user(bay_area, movement[i]) for i in tqdm(range( population )) ])

In [ ]:
# TEST ACCURACY:
def plot_accuracy(scheme, epsilon):
    USGS = semi_honest( bay_area, go_bears, scheme(epsilon) )
    return USGS.accuracy()

exogenous  = numpy.arange(0.02, 1.00, 0.01)
for name, scheme in tqdm( privacy_schemes.items() ):
    endogenous = [ plot_accuracy(scheme, epsilon) for epsilon in tqdm(exogenous, desc = "Epsilon") ]
    plot.plot(exogenous, endogenous, label = name)

plot.legend()
plot.show()

In [ ]:
# TEST PRIVACY
exogenous = range(bay_area.epochs)

for name, scheme in tqdm( privacy_schemes.items() ):    
    USGS = semi_honest( bay_area, go_bears, scheme(0.02) )
    endogenous = USGS.average_predictive_power()
    
    plot.plot(exogenous, endogenous, label = name)

plot.legend()
plot.show()

In [ ]:
USGS = semi_honest( bay_area, go_bears, geo_indistinguishable(0.02) )

plot.scatter( * zip(* bay_area.spaces), s = USGS.users[0].location_density * 100 )
plot.scatter( * zip(* bay_area.spaces), s = USGS.timeseries_inference(0)[-1] * 100 )
plot.axis("equal")
plot.show()

In [ ]:
USGS = semi_honest( bay_area, go_bears, gaussian(0.02) )

plot.scatter( * zip(* bay_area.spaces), s = USGS.users[0].location_density * 100 )
plot.scatter( * zip(* bay_area.spaces), s = USGS.timeseries_inference(0)[-1] * 100 )

plot.axis("equal")
plot.show()

In [ ]:
USGS = semi_honest( bay_area, go_bears, cloaking(0.02) )

plot.scatter( * zip(* bay_area.spaces), s = USGS.users[0].location_density * 100 )
plot.scatter( * zip(* bay_area.spaces), s = USGS.timeseries_inference(0)[-1] * 100 )

plot.axis("equal")
plot.show()